# MOwNiT - układy równań liniowych

<h3> Przydatne linki: </h3>

- CPP: https://en.cppreference.com/w/

- Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
- Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney* str. 245, pełny -pseudokod: str. 252
- Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
- Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
- Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
- Faktoryzacja Cholesky'ego: https://en.wikipedia.org/wiki/Cholesky_decomposition, K.C. str. 305
- Wyznacznik macierzy: https://pl.wikipedia.org/wiki/Wyznacznik

**Dodatkowe źródła przydatne przy implementacjach**
- Rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
- Rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).

<h3> Zadania </h3>

**Zadanie 1** 
W załączonym do laboratorium kodzie napisz funkcje realizujące dodawanie oraz mnożenie macierzy.

In [1]:
#include <vector>
#include <iostream>
#include <cmath>


template <typename T> class AGHMatrix 
{
private:
    std::vector<std::vector<T>> matrix;
    unsigned rows;
    unsigned cols;

public:
    AGHMatrix(const std::vector<std::vector<T>>& matrix);
    AGHMatrix(unsigned _rows, unsigned _cols, const T& _initial);
    AGHMatrix(const AGHMatrix<T>& rhs);
    virtual ~AGHMatrix() = default;

    // Operator overloading, for "standard" mathematical matrix operations                                                                                                                                                          
    AGHMatrix<T>& operator=(const AGHMatrix<T>& rhs);

    // Matrix mathematical operations                                                                                                                                                                                               
    AGHMatrix<T> operator+(const AGHMatrix<T>& rhs);
    AGHMatrix<T> operator*(const AGHMatrix<T>& rhs);

    // Access the individual elements                                                                                                                                                                                               
    T& operator()(const unsigned& row, const unsigned& col);
    const T& operator()(const unsigned& row, const unsigned& col) const;
    
    // Printing matrix
    std::ostream& operator<<(const AGHMatrix<T>& matrix);

    // Access the row and column sizes                                                                                                                                                                                              
    unsigned get_rows() const;
    unsigned get_cols() const;
    
    // Zad2
    bool is_symmetric();
    AGHMatrix<T> get_minor(int col, int row);
    T get_determinant();
    void transpose();
};

In [2]:
// Addition of two matrices                                                                                                                                                   
template<typename T>
AGHMatrix<T> AGHMatrix<T>::operator+(const AGHMatrix<T>& rhs) 
{
  // Task 1 - implement addition of two matrices
    if(this->get_rows()!=rhs->get_rows() || this->get_cols()!=rhs->get_cols()){
        std::cout<<"Nie poprawne rozmiary maciezy.";
        return 1;
    }
    
    int rows = this->get_rows();
    int colls = this->get_cols();
    
    for(int i=0; i<rows; i++){
        for(int j=0; j<cols; j++){
            this->matrix[i][j]+=rhs->matrix[i][j];
        }
    }
}

// Multiplication of this matrix and other                                                                                                                           
template<typename T>
AGHMatrix<T> AGHMatrix<T>::operator*(const AGHMatrix<T>& rhs) 
{
  // Task 1 - implement multiplication of two matrices
    if(this->get_cols()!=rhs->get_rows()){
        std::cout<<"Nie poprawne rozmiary maciezy.";
        return 1;
    }
    
    AGHMatrix<T> output (this->get_rows(), rhs.get_cols(), 0);
    
    int rows = output->get_rows();
    int cols = output->get_cols();
    
    for(int i=0; i<rows; i++){
        for(int j=0; j<cols; j++){
            for(int k=0; k<this->get_rows(); k++){
                output->matrix[i][j]+=this->matrix[i][k]*rhs->matrix[k][j];
            }
        }
    }
    
    return output;
}

**Uwaga** W poniższych zadania można korzystać z kodu laboratoryjnego dot. macierzy, albo stworzyć własną klasę/strukturę macierzy, na której będą realizowane dalsze zadania.

**Zadanie 2**  Zaimplementuj: 
    1. Funkcję/metodę, która sprawdzi czy macierz jest symetryczna. 
    2. Funkcję/metodę, która obliczy wyznacznik macierzy.
    3. Metodę transpose().

In [3]:
/* --IMPELEMENTACJA-W-AGHMATRIX.CPP--
template<typename T>
bool AGHMatrix<T>::is_symmetric(){
    if(this->get_rows()!=this->get_cols()){
        return false;
    }
    
    int rows = this->get_rows();
    int cols = this->get_cols();
    
    for(int i = rows; i>=0; i--){
        for(int j=0; j<i; j++){
            if(!(this->matrix[rows-i][j]!=this->matrix[j][rows-i])){
                return false;
            }
        }
    }
    
    return true;
}

template<typename T>
AGHMatrix<T> AGHMatrix<T>::get_minor(int col, int row){
    int rows = this->get_rows()-1;
    int cols = this->get_cols()-1;
    
    AGHMatrix<T> matrix_minor (rows, cols, 0);
    
    int r=0;
    int c=0;
    for(int i = 0; i<rows; i++){
        if(i!=row){
            for(int j = 0; j<cols; j++){
                if(j!=col){
                    matrix_minor->matrix[r][c]=this->matrix[i][j];
                    c++;
                }
            }
            r++;
        }
    }
    
    return matrix_minor;
}


template<typename T>
T AGHMatrix<T>::get_determinant(){
    if((this->get_rows()!=this->get_cols()||this->get_rows()<1)){
        std::cout<<"Invalid matrix shape.";
        return 1;
    }
    if(this->get_rows()==1){
        return this->matrix[0][0];
    }   
    
    if(this->get_rows()==2){
        return this->matrix[0][0]*this->matrix[1][1]-this->matrix[0][1]*this->matrix[1][0];
    }
    
    T result = 0;
    int rows = this->get_rows();
    int cols = this->get_cols();
    
    for(int i=0; i<rows; i++){
        for(int j=0; j<cols; j++){
            AGHMatrix<T> minor = get_minor(i, j);
            result += minor.get_determinant()*(pow(-1, i+j));
        }
    }
    
    return result;
}

template<typename T>
void AGHMatrix<T>::transpose(){
    int rows = this->get_rows();
    int cols = this->get_cols();
    
    AGHMatrix<T> transposed (cols, rows, 0);
    
    for(int i=0; i<rows; i++){
        for(int j=0; j<cols; j++){
            transposed->matrix[j][i]=this->matrix[i][j];
        }
    }
    
    this = transposed;
}
*/

**Zadanie 3**  Proszę zaimplementować algorytm faktoryzacji LU macierzy (można to zrobić przy użyciu kodu dostarczonego do laboratorium lub stworzyć własną strukturę macierzy i na niej działać). Algorytm przetestować na przykładzie z [wikipedii](https://pl.wikipedia.org/wiki/Metoda_LU) lub korzystając z poniższego kodu.  

In [1]:
#pragma cling load("aghmatrix.h")
// initialize matrix using specified values
std::vector<std::vector<double>> init_LU {{ 5.0, 3.0, 2.0 }, 
                                          { 1.0, 2.0, 0.0 }, 
                                          { 3.0, 0.0, 4.0 }};

// Jeśli się korzysta z implementacji laboratoryjnej
AGHMatrix<double> LU(init_LU);
int rows = LU.get_rows();
AGHMatrix<double> L (rows, rows, 0);
AGHMatrix<double> U (rows, rows, 0);
for(int i=0; i<rows; i++){
    L(i,i)=1;
}

for(int i=0; i<rows; i++){
    for(int j=i; j<rows; j++){
        double sum=0;
        for(int k=0; k<i; k++){
            sum+=L(i,k)*U(k,i);
        }
        U(i,j)=(LU(i,j)-sum);
    }
    for(int j=i+1; j<rows; j++){
        int sum=0;
        for(int k=0; k<i-1; k++){
            sum+=L(j,k)*U(k, i);
        }
        L(j, i)=(LU(j,i)-sum)/U(i,i);
    }
}

std::cout<<U;
std::cout<<L;


5, 3, 2, 
0, 1.4, -0.6, 
0, 0, 2.8, 

1, 0, 0, 
0.2, 1, 0, 
0.6, 0, 1, 



**Zadanie 4**  Proszę zaimplementować algorytm faktoryzacji Cholesky'ego macierzy. Jego test można przeprowadzić analogicznie do poprzedniego zadania i oprzeć o przykład z [wikipedii](https://en.wikipedia.org/wiki/Cholesky_decomposition). Po zakończeniu tego zadania proszę porównać oba algorytmy faktoryzacyjne i opisać różnice w ich konstrukcji.

In [1]:
#pragma cling load("aghmatrix.h")
// initialize matrix using specified values
std::vector<std::vector<double>> init_cholesky {{ 4.0, 12.0, -16.0 }, 
                                                { 12.0, 37.0, -43.0 }, 
                                                { -6.0, -43.0, 98.0 }};

// Jeśli się korzysta z implementacji laboratoryjnej
AGHMatrix<double> A(init_cholesky);

int rows = A.get_rows();

AGHMatrix<double> L(rows, rows, 0);

L(0,0)=sqrt(A(0,0));

for(int i=1; i<rows; i++){
    L(i,0)=A(i,0)/L(0,0);
}

for(int i=1; i<rows; i++){
    for(int j=1; j<=i; j++){
        double sum=0;
        if(i==j){
            for(int k=0; k<j; k++){
                sum+=pow(L(i,k),2);
            }
            L(i,j)=sqrt(A(i,j)-sum);
        } else{
            i--;
            for(int k=0; k<i; k++){
                sum+=L(i,k)*L(j,k);
            }
            L(i,j)=(A(i,j)-sum)/L(i,i);
            i++;
        }
    }
}

std::cout<<L;


2, 0, 0, 
6, 1, 0, 
-3, 0, 9.43398, 



**Zadanie 5**  Proszę napisać funkcję (lub klasę wraz z metodami), która realizuje eliminacje Gaussa. Proszę starannie opisać kod, który ją realizuje. Test algorytmu jest najłatwiej zrealizować przy pomocy języka python oraz pakietu numpy (poniższy kod). 

(*) Dla chętnych - można napisać prosty TestCase, który porówna dwie macierze. Poprawną najlepiej znalaźć przy pomocy pythona. Środowisk testowych w C++ jest kilka - ja polecam GoogleTest.   


In [2]:
// A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
//                [2.2660, 1.9950,  1.2120, 8.0080],
//                [8.8500, 5.6810,  4.5520, 1.3020],
//                [6.7750, -2.253,  2.9080, 3.9700]])

// b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

// x = np.linalg.solve(A, b)

// Checking
// np.allclose(np.dot(A, x), b)
// Rozwiązanie w pliku Gauss.ipynb

**Zadanie 6** Implementacja metody Jackobiego - tworzenie i wymagania analogicznie do Zad.4.

In [6]:
#pragma cling load("aghmatrix.h")
// initialize matrix using specified values
std::vector<std::vector<double>> init_jacobi {{ 4.0, 12.0, -16.0 }, 
                                              { 12.0, 37.0, -43.0 }, 
                                              { -6.0, -43.0, 98.0 }};

AGHMatrix<double> A(init_jacobi);

std::vector<double> X;

for(int i=0; i<A.get_rows(); i++){
    X.push_back(0);
}

for(int j=0; j<100; j++){
    for(int i=0; i<A.get_rows(); i++){
        double sum = 0;
        for(int k=0; k<A.get_cols()-1; k++){
            if(k!=i)
                sum+=X[k]*A(i, k);
        }
        X[i]=(A(i, A.get_cols()-1)-sum)/A(i,i);
    }
}

for(int i=0; i<A.get_rows(); i++){
    std::cout<<X[i]<<'\t';
}

input_line_13:4:34: error: redefinition of 'init_jacobi'
std::vector<std::vector<double>> init_jacobi {{ 4.0, 12.0, -16.0 }, 
                                 ^
input_line_9:4:34: note: previous definition is here
std::vector<std::vector<double>> init_jacobi {{ 4.0, 12.0, -16.0 }, 
                                 ^


Interpreter Error: 